In [63]:
import pandas as pd
from googletrans import Translator
from tqdm import tqdm
from dotenv import load_dotenv
from pathlib import Path
import json, os

translator = Translator()
folder_path = "src/pointx"
change_type = { "string" : "text",
                "int" : "number",
                "bigint": "number",
                "decimal(27,2)" : "number",
                "double" : "number",
                "timestamp" : "text",
                "date" : "text"
}
schema_desc_path = os.path.join(folder_path,"ETL Mapping & Data Dict - PointX (1).xlsx")
# dotenv_path = Path('.env')
# load_dotenv(dotenv_path=dotenv_path)

# pointx_keymatrix_dly	Table

In [ ]:
df = pd.read_excel(schema_desc_path, sheet_name='14')
df.columns = df.iloc[17,:]
df = df.iloc[18:,:].reset_index(drop=True)
df.columns.name = None
df.head()

In [ ]:
col_types = {}
col_descs = {}
table_name = df['Table'].unique().tolist()[0]
table_desc = """The Key Matrix Dashboard Design table provides a detailed overview of dashboard-related database columns, 
including data types, status indicators, descriptions, conditions, business logic, and sample data, 
enabling a comprehensive understanding of the data structure for effective dashboard design."""

for i, row in tqdm(df.iterrows()):
    col_name = row['Column']
    data_type = change_type[row['Data Type'].lower()]
    desc = translator.translate(row['Description'], dest='en').text

    col_types[col_name] = data_type
    col_descs[col_name] = desc


In [ ]:
schema_desc = {
    "table": table_name,
    "description": table_desc,
    "columns": col_descs
}

# with open(os.path.join(folder_path, "pointx_keymatrix_dly_schema_description.json"),'w') as f:
#     json.dump(schema_desc, f, indent=4)

# with open(os.path.join(folder_path, "pointx_keymatrix_dly_columns_type.json"),'w') as f:
#     json.dump(col_types, f, indent=4)

In [ ]:
# with open(os.path.join(folder_path, "pointx_keymatrix_dly_schema_description.json"),'r') as f:
#     col_descs = json.load(f)

# pointx_cust_mly Table

In [64]:
df = pd.read_csv("src/pointx/pointx_cust_mly_datatype.csv")
col_types = df[['col_name', 'data_type']].set_index('col_name')['data_type'].to_dict()

for c in col_types:
    col_types[c] = change_type[col_types[c]]
    
with open(os.path.join(folder_path, "pointx_cust_mly_type.json"),'w') as f:
    json.dump(col_types, f, indent=4)

In [ ]:
df = pd.rea

# pointx_fbs_rpt_dly Table

In [51]:
table_name = "pointx_fbs_rpt_dly"
table_desc = """Table records user interactions with the PointX app daily, capturing events such as app opens and deletions, 
providing key insights into user behavior, app version usage, and device characteristics """

df = pd.read_csv("src/pointx/pointx_fbs_rpt_dly_description.csv")
col_descs = df.set_index('Column')['Description'].to_dict()

In [52]:
schema_desc = {
    "table": table_name,
    "description": table_desc,
    "columns": col_descs
}

# with open(os.path.join(folder_path, "pointx_fbs_rpt_dly_description.json"),'w') as f:
#     json.dump(schema_desc, f, indent=4)

## Let's filtering

In [ ]:
import os, json, sqlparse, re, nltk
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
from nltk.stem import WordNetLemmatizer
from sql_metadata import Parser

src_folder = "src/pointx"
schema_description_file = "pointx_keymatrix_dly_schema_description.json"
with open(os.path.join(src_folder, schema_description_file)) as f:
    db = json.load(f)
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
lemmanizer = WordNetLemmatizer()

In [ ]:
schema_vector = []

schema_emb = {}
table_name = db['table']
table_description = db['description']
schema_emb[table_name] = model.encode(table_description).tolist()
columns = list(db['columns'].keys())
for col in columns:
    column_description = db['columns'][col]
    schema_emb[col] = model.encode(column_description).tolist()
schema_vector.append(schema_emb)

In [ ]:
def column_from_question(question,used_table_col = {}, default_score=0.6):
    # question_tokens = [token.lower() for token in tokenizer.tokenize(question)]
    question_tokens = [lemmanizer.lemmatize(token.lower()) for token in question.split()]
    # print(question_tokens)
    for table in schema_vector:
        max_score = default_score
        for token in question_tokens:
            cols = [ key.lower() for key in table.keys()]
            table_name = cols.pop(0)
            if token == table_name: 
                max_score = 1.0
                # plus the score of columns in exact match with table name
                if used_table_col.get(token) is not None: 
                    for key in used_table_col[token]: 
                        used_table_col[token].update({key: used_table_col[token][key] + 0.1})
            # exact match table and column
            if token in cols: 
                used_table_col.setdefault(table_name, {}).update({token : max_score})

    return used_table_col

In [ ]:
def filter_tables_by_description(question, column_threshold = 0.4, table_threshold = 0.2, filter_tables = True):
    question_emb = model.encode(question)
    used_schema = {}
    for i in range(len(schema_vector)):
        table_name = list(schema_vector[i].keys())[0]

        table_description_vector = schema_vector[i][table_name]
        if filter_tables and util.cos_sim(table_description_vector, question_emb) < table_threshold: continue
        
        used_col = {}
        for col, vec in schema_vector[i].items():
            if col == table_name: continue
            score = round(float(util.cos_sim(vec, question_emb)),2)
            if score > column_threshold:
                # column_description = [dbs[i]['columns'][col] for i in range(len(dbs)) if dbs[i]['table'] == table_name][0]
                # print(f"{table_name} - {col} : {score}\nDescription : {column_description}\n")
                used_col.update({col: score})
        if len(used_col) > 0: used_schema[table_name] = used_col
    return used_schema

In [ ]:
question = "user open app"
selected_table_column = filter_tables_by_description(question, column_threshold = 0.3, filter_tables = False)
selected_table_column = column_from_question(question, used_table_col=selected_table_column)
selected_table_column